In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831126 sha256=27dfe6b73c2f9f81fd16bfccf1aa18e8a8db1d05ade16d3585cc4844db17edfe
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter("ignore")
seed = 2354
np.random.seed(seed=seed)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [6]:
games_df = pd.read_csv("/kaggle/input/game-recommendations-on-steam/games.csv")
users_df = pd.read_csv("/kaggle/input/game-recommendations-on-steam/users.csv")
games_meta_df = pd.read_json("/kaggle/input/game-recommendations-on-steam/games_metadata.json", lines=True)
iteractions_df = pd.read_csv("/kaggle/input/game-recommendations-on-steam/recommendations.csv")

In [7]:
games_meta_df.head()

,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,,[Action]
2,113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


# Предобработка

In [11]:
iteractions_df["app_id"].value_counts()

app_id
440        319492
252490     270684
1091500    226414
730        219737
570        216914
            ...  
1814870         1
1136110         1
1771540         1
2094300         1
498940          1
Name: count, Length: 37610, dtype: int64

In [17]:
temp = iteractions_df["app_id"].value_counts()
filt_index = temp[temp >= 20].index

In [18]:
iteractions_df = iteractions_df[iteractions_df["app_id"].isin(filt_index)]

In [19]:
global_train, global_test = (
    train_test_split(
        iteractions_df
        ,test_size = .2
        ,random_state = seed
        ,stratify = iteractions_df["app_id"])
)